# Installs

## wandb

You will need to fetch your api key from wandb.ai

In [1]:
!pip install wandb -q

## Misc

This may take a while

In [2]:
!pip install torchsummaryX
!pip install slugify
!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## imports

In [3]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from PIL import Image
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import wandb
import copy
from pathlib import Path
import sys

import warnings
warnings.filterwarnings('ignore')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
!mkdir '/content/glove'
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -qo 'glove.6B.zip' -d '/content/glove'

# Google Drive

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Setting

In [35]:
config = {
    "num_labels" : 2, # True or False Classification
    "lr" : 2e-3,
    "epochs" : 50,
    'batch_size' : 4,
    #'LR scheduler': 'CosineAnnealingLR',
    'LR scheduler': 'ReduceLROnPlateau',
                'scheduler factor': 0.5,
    'scheduler threshold': 0.01,
            'scheduler patience': 5,
   #'scheduler Tmax': 0,
    } # Feel free to add more items here

# Read Data

In [36]:
# remeber to change the path if you are not on google colab and directly uploading
train_path = '/content/train2.tsv'
test_path = '/content/test2.tsv'
val_path = '/content/val2.tsv'

In [37]:
train_df = pd.read_csv(train_path, sep="\t", header=None)
test_df = pd.read_csv(test_path, sep="\t", header=None)
val_df = pd.read_csv(val_path, sep="\t", header=None)

# Fill nan (empty boxes) with 0
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
val_df = val_df.fillna(0)

train = train_df.values
test = test_df.values
val = val_df.values

In [38]:
# label: ground truth results from politifacts
labels = {'train':[train[i][2] for i in range(len(train))], 'test':[test[i][2] for i in range(len(test))], 'val':[val[i][2] for i in range(len(val))]}
# Short Statement
statements = {'train':[train[i][3] for i in range(len(train))], 'test':[test[i][3] for i in range(len(test))], 'val':[val[i][3] for i in range(len(val))]}
# Topic of Statement
subjects = {'train':[train[i][4] for i in range(len(train))], 'test':[test[i][4] for i in range(len(test))], 'val':[val[i][4] for i in range(len(val))]}
# Speaker
speakers = {'train':[train[i][5] for i in range(len(train))], 'test':[test[i][5] for i in range(len(test))], 'val':[val[i][5] for i in range(len(val))]}
# Speaker job or title
jobs = {'train':[train[i][6] for i in range(len(train))], 'test':[test[i][6] for i in range(len(test))], 'val':[val[i][6] for i in range(len(val))]}
# State of Relevance
states = {'train':[train[i][7] for i in range(len(train))], 'test':[test[i][7] for i in range(len(test))], 'val':[val[i][7] for i in range(len(val))]}
# party affiliation
affiliations = {'train':[train[i][8] for i in range(len(train))], 'test':[test[i][8] for i in range(len(test))], 'val':[val[i][8] for i in range(len(val))]}
# total history of speaker(count of barely true, false, half true, mostly true, pants on fire respectively)
credits = {'train':[train[i][9:14] for i in range(len(train))], 'test':[test[i][9:14] for i in range(len(test))], 'val':[val[i][9:14] for i in range(len(val))]}
# venue of statement
contexts = {'train':[train[i][14] for i in range(len(train))], 'test':[test[i][14] for i in range(len(test))], 'val':[val[i][14] for i in range(len(val))]}
# verdict justification from politifacts
justification = {'train':[train[i][15] for i in range(len(train))], 'test':[test[i][15] for i in range(len(test))], 'val':[val[i][15] for i in range(len(val))]}

In [39]:
# currently only do 2 way classfication & simplify 6 way label into true and false
# convert label to 2 hot based on verdict label
if config["num_labels"] ==2:
  def onehot(label):
    label_onehot = [0]*len(label)
    for i in range(len(label)):
      if label[i] =='true' or label[i] =='mostly-true' or label[i] =='half-true':
        label_onehot[i] = [1,0]
      elif label[i] =='barely-true' or label[i] =='false' or label[i] =='pants-fire':
        label_onehot[i] = [0,1]
      else:
        print('Unexpected Label. Set vector to [0]')
    return label_onehot

In [40]:
# Convert to one hot
label_onehot = {'train':onehot(labels['train']), 'test':onehot(labels['test']), 'val':onehot(labels['val'])}

In [41]:
# Meta data
metadata = {'train':[0]*len(train), 'val':[0]*len(val), 'test':[0]*len(test)}

for i in range(len(train)):
    subject = subjects['train'][i]
    if subject == 0:
        subject = 'None'

    speaker = speakers['train'][i]
    if speaker == 0:
        speaker = 'None'

    job = jobs['train'][i]
    if job == 0:
        job = 'None'

    state = states['train'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['train'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['train'][i]
    if context == 0 :
        context = 'None'
    if i == 0:
      print(subject)
    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context
    if i == 0:
      print(meta)
    metadata['train'][i] = meta

for i in range(len(val)):
    subject = subjects['val'][i]
    if subject == 0:
        subject = 'None'

    speaker = speakers['val'][i]
    if speaker == 0:
        speaker = 'None'

    job = jobs['val'][i]
    if job == 0:
        job = 'None'

    state = states['val'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['val'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['val'][i]
    if context == 0 :
        context = 'None'

    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context

    metadata['val'][i] = meta

for i in range(len(test)):
    subject = subjects['test'][i]
    if subject == 0:
        subject = 'None'

    speaker = speakers['test'][i]
    if speaker == 0:
        speaker = 'None'

    job = jobs['test'][i]
    if job == 0:
        job = 'None'

    state = states['test'][i]
    if state == 0:
        state = 'None'

    affiliation = affiliations['test'][i]
    if affiliation == 0:
        affiliation = 'None'

    context = contexts['test'][i]
    if context == 0 :
        context = 'None'

    meta = subject + ' ' + speaker + ' ' + job + ' ' + state + ' ' + affiliation + ' ' + context

    metadata['test'][i] = meta

abortion
abortion dwayne-bohac State representative Texas republican a mailer


In [42]:
# Credit score calculation
# barely true weighs 0.75, false weighs 0.9, half true weigh 0.5, mostly true weigh 0.2, pants on fire weigh 1
credit_score = {'train':[0]*len(train), 'val':[0]*len(val), 'test':[0]*len(test)}
for i in range(len(train)):
    credit = credits['train'][i]
    if sum(credit) == 0:
        score = 0.5
    else:
        score = (credit[3]*0.2 + credit[2]*0.5 + credit[0]*0.75 + credit[1]*0.9 + credit[4]*1)/(sum(credit))
    credit_score['train'][i] = [score for i in range(2304)]

for i in range(len(val)):
    credit = credits['val'][i]
    if sum(credit) == 0:
        score = 0.5
    else:
        score = (credit[3]*0.2 + credit[2]*0.5 + credit[0]*0.75 + credit[1]*0.9 + credit[4]*1)/(sum(credit))
    credit_score['val'][i] = [score for i in range(2304)]

for i in range(len(test)):
    credit = credits['test'][i]
    if sum(credit) == 0:
        score = 0.5
    else:
        score = (credit[3]*0.2 + credit[2]*0.5 + credit[0]*0.75 + credit[1]*0.9 + credit[4]*1)/(sum(credit))
    credit_score['test'][i] = [score for i in range(2304)]


# Dataset and Dataloader

In [43]:

# Loading the statements
X_train = statements['train']
y_train = label_onehot['train']

X_val = statements['val']
y_val = label_onehot['val']


X_test = statements['test']
y_test = label_onehot['test']

# Loading the justification
X_train_just = justification['train']

X_val_just = justification['val']


X_test_just = justification['test']


# Loading the meta data
X_train_meta = metadata['train']
X_val_meta = metadata['val']
X_test_meta = metadata['test']

# Loading Credit scores

X_train_credit = credit_score['train']
X_val_credit = credit_score['val']
X_test_credit = credit_score['test']

In [44]:
max_seq_length_stat = 64
max_seq_length_just = 256
max_seq_length_meta = 32

### Train Data

In [45]:
class TextDataset(torch.utils.data.Dataset):



    def __init__(self, xy_list ,transform=None): 
        '''
        Initializes the dataset.

        '''

        # Load the xy list

        self.x_y_list = xy_list
        self.length = len(xy_list[0])
        for i in range(self.length):
        #   Load in each statement and tokenize
            #print(self.x_y_list[0][i])
            tokenized_stat = tokenizer.tokenize(self.x_y_list[0][i])
            if len(tokenized_stat) > max_seq_length_stat:
              # clip if the statement is too long
                tokenized_stat = tokenized_stat[:max_seq_length_stat]

            # convert statement to ids
            ids_stat  = tokenizer.convert_tokens_to_ids(tokenized_stat)
            # pad the statement to given length
            padding = [0] * (max_seq_length_stat - len(ids_stat))

            ids_stat += padding
            # sanity check
            assert len(ids_stat) == max_seq_length_stat
            
            #if i == 1:
            #  print(ids_stat)
            ids_stat = torch.tensor(ids_stat)
            
            if self.x_y_list[1][i] == 0:
                self.x_y_list[1][i] = 'No justification'

            #print(self.x_y_list[1][i])
            tokenized_just = tokenizer.tokenize(self.x_y_list[1][i])
            if len(tokenized_just) > max_seq_length_just:
              # clip if the statement is too long
                tokenized_just = tokenized_just[:max_seq_length_just]

            # convert statement to ids
            ids_just  = tokenizer.convert_tokens_to_ids(tokenized_just)
            # pad the statement to given length
            padding = [0] * (max_seq_length_just - len(ids_just))

            ids_just += padding
            # sanity check
            assert len(ids_just) == max_seq_length_just

            #if i == 1:
            #  print(ids_just)
            ids_just = torch.tensor(ids_just)

            #print(self.x_y_list[2][i])
            tokenized_meta = tokenizer.tokenize(self.x_y_list[2][i])
            if len(tokenized_meta) > max_seq_length_meta:
              # clip if the statement is too long
                tokenized_meta = tokenized_meta[:max_seq_length_meta]

            # convert statement to ids
            ids_meta  = tokenizer.convert_tokens_to_ids(tokenized_meta)
            # pad the statement to given length
            padding = [0] * (max_seq_length_meta - len(ids_meta))

            ids_meta += padding
            # sanity check
            assert len(ids_meta) == max_seq_length_meta

            ids_meta = torch.tensor(ids_meta)
            
            credit_scr = torch.tensor(self.x_y_list[3][i]) # Credit score

            #if i == 1:
            #  print(credit_scr)

            label = torch.from_numpy(np.array(self.x_y_list[4][i]))

            #if i == 1:
            #  print(label)

            self.x_y_list[0][i] = ids_stat
            self.x_y_list[1][i] = ids_just
            self.x_y_list[2][i] = ids_meta
            self.x_y_list[3][i] = credit_scr
            self.x_y_list[4][i] = label







    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        

        ids_stat = self.x_y_list[0][ind] 
        ids_just = self.x_y_list[1][ind]
        ids_meta = self.x_y_list[2][ind] 
        credit_scr = self.x_y_list[3][ind] 
        label = self.x_y_list[4][ind]

        return ids_stat, ids_just, ids_meta, credit_score, label



### Data - Hyperparameters

In [46]:
BATCH_SIZE = config['batch_size'] # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation


### Data loaders

In [47]:
# get me RAMMM!!!! 
import gc 
gc.collect()
X_train_meta[0]

'abortion dwayne-bohac State representative Texas republican a mailer'

In [48]:
# Create objects for the dataset class
#train_data = TextDataset([X_train[:100], X_train_just[:100], X_train_meta[:100], X_train_credit[:100], y_train[:100]])
train_data = TextDataset([X_train, X_train_just, X_train_meta, X_train_credit, y_train])
val_data = TextDataset([X_val, X_val_just, X_val_meta, X_val_credit, y_val]) 
test_data = TextDataset([X_test, X_test_just, X_test_meta, X_test_credit, y_test]) 

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=BATCH_SIZE, pin_memory= True,
                                           shuffle= True)
val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                           batch_size=BATCH_SIZE, pin_memory= True,
                                           shuffle= True)
test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2,
                                           batch_size=BATCH_SIZE, pin_memory= True,
                                           shuffle= True)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  4
Train dataset samples = 10240, batches = 2560
Val dataset samples = 1284, batches = 321
Test dataset samples = 1267, batches = 317


In [49]:
# sanity check
for data in train_loader:
    x, y, z, m, n = data
    print(x.shape, y.shape, z.shape)
    break 

torch.Size([4, 64]) torch.Size([4, 2])


# Model Config

## Basic

## INIT

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                           Kernel Shape     Output Shape    Params Mult-Adds
Layer                                                                       
0_embedding.Conv1d_0       [15, 256, 5]  [64, 256, 1664]   19.456k  31.9488M
1_embedding.BatchNorm1d_1         [256]  [64, 256, 1664]     512.0     256.0
2_embedding.ReLU_2                    -  [64, 256, 1664]         -         -
3_lstm                                -     [80474, 256]  526.336k  524.288k
4_classification.Linear_0     [256, 41]   [64, 1664, 41]   10.537k   10.496k
5_logSoftmax                          -   [64, 1664, 41]         -         -
------------------------------------------------------------------------------
                         Totals
Total params           556.841k
Trainable params       556.841k
Non-trainable params        0.0
Mult-Adds             32.48384M


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding.Conv1d_0,"[15, 256, 5]","[64, 256, 1664]",19456.0,31948800.0
1_embedding.BatchNorm1d_1,[256],"[64, 256, 1664]",512.0,256.0
2_embedding.ReLU_2,-,"[64, 256, 1664]",NaN,NaN
3_lstm,-,"[80474, 256]",526336.0,524288.0
4_classification.Linear_0,"[256, 41]","[64, 1664, 41]",10537.0,10496.0
5_logSoftmax,-,"[64, 1664, 41]",NaN,NaN


# Training Config

In [ ]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

input_shape torch.Size([64, 1687, 15])
y_shape torch.Size([64, 209])
input_lengths_shape torch.Size([64])
ly_shape torch.Size([64])
out:torch.Size([1685, 64, 41])
out_len:torch.Size([64])
loss: 30.570667266845703
lev-distance: 506.84375


[' ',
 '-',
 'G',
 'f',
 'm',
 '@',
 'r',
 'u',
 'n',
 'i',
 'W',
 'v',
 'U',
 'o',
 'a',
 'R',
 'h',
 'z',
 'k',
 'C',
 'w',
 'e',
 'Z',
 't',
 'E',
 'y',
 'A',
 'b',
 'p',
 'T',
 'D',
 'c',
 'g',
 'l',
 'j',
 'O',
 'S',
 'd',
 'Y',
 's',
 'I']

In [ ]:
wandb.login(key="")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [ ]:
run = wandb.init(
    name = "project-1", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    #id = '2lq40515', #Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    #project="hw3p2", ### Project should be created in your wandb account 
    config=config, ### Wandb Config for your run
    entity="the-spinning-top"
)

# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

### Training Setup

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:
CHECKPOINT_PATH = 'checkpoint.pth'
#print(wandb.run.resumed)
if wandb.run.resumed:
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    val_dist = checkpoint['val_dist']

### Train Loop


Epoch 1/50: 
 Train Loss 3.5837	 Learning Rate 0.0020


Val Loss 2.7885	 Val Dist 71.5003
Saving model



Epoch 2/50: 
 Train Loss 2.2803	 Learning Rate 0.0020


Val Loss 1.9820	 Val Dist 49.3027
Saving model



Epoch 3/50: 
 Train Loss 1.7336	 Learning Rate 0.0020


...


Epoch 28/50: 
 Train Loss 0.8520	 Learning Rate 0.0020


Val Loss 0.9554	 Val Dist 21.4545



Epoch 29/50: 
 Train Loss 0.8430	 Learning Rate 0.0020


Val Loss 0.9501	 Val Dist 21.2943
Saving model


Train:  27%|███████████████████████████████▏                                                                                    | 120/446 [00:23<01:02,  5.19it/s, loss=0.8322, lr=0.0020]

# Generate Predictions

0       YkTdC-gmtjkOgrtjdcicCnwsWCdab-jumkkYtfuE-gtElD...
1       vwt@SjjATiYlOfgekScgcjtGWmGDoSviakm-chrrG-ckvR...
2       ksuCnREU-iOeilCivhtldAjTthGkRkwwtSiavlsRSnnWsg...
3       jUkAjnATe-uaodmRrp-c-GsoGDkEhuSkUavkUhdpjjgDhD...
4       jpdcumtfICDDDWigtgSuobt-IvkbSAiOyRybd@GYubYomT...
                              ...                        
2615    YYkkmTAjhcYTyeGhbUImko@WtyyuTEUigAGkGEpnUiitI-...
2616    SiyEkECRkpusUSaOaSSYTIpvWonU@ayt-iWjpykdeAltpy...
2617    toEbcWrfTegCIudTRkugnm@WgkpCtfmoUhupsDnlUm-TIh...
2618    AdGCtwt@Dmloah@mIdkieoAjOoeklECGrGtSsRwkgppCtD...
2619    -ibspmWgT@OtUDnEUYYsRkudRdoblEnillpbbCktRvTmiI...
Name: label, Length: 2620, dtype: object


100% 199k/199k [00:00<00:00, 221kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)